# **1. Corpus**

## Crawl URLs to extract all internal links 

**XENU Link Sleuth**  
https://home.snafu.de/tilman/xenulink.html 

*Le logiciel XENU Link Sleuth a finalement été retenu pour cette tâche* 

## Scrape textual data from crawled URLs
**BeautifulSoup HTML Parser**  
Réf : https://realpython.com/python-web-scraping-practical-introduction/

In [1]:
import os
from pandas import *

corpus_global = []
base_path = '../03-corpus/1-crawler/'
for file in os.listdir(base_path):
    if file.endswith('.csv'):
        with open(base_path + file, encoding = 'utf-8',) as f:
            corpus_global.append({'acteur': file[:-4]}) 

In [2]:
import requests, re, ssl, os, sys, pandas as pd
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry

def getTextHTML(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text

    soup = BeautifulSoup(html, "html.parser")
    tags_to_remove = ['head', 'header', 'script', 'footer', 'nav']#, 'form'] # Enlever 'form' pour le site du CHU Ste Justine

    # Classes CSS spécifiques aux différents sites
    attr_to_remove = ['div[class="contenu-fluide piv"]', 'div[role="navigation"]', 'div[class="section__wrapper section__wrapper--padding tac grid--inline-block"]', 'div[id="slidebox"]',
    'div[class="col-md-12 mise-a-jour"]', 'h1[class="sr-only"]', 'a[class="cd-top js-cd-top"]', 'div[class="nocontent"]', 'p[class="footer-lien-resonances"]', 'p[class="suivre"]',
    'section[class="field field-name-field-date-de-mise-jour field-type-datetime field-label-inline clearfix view-mode-full"]', 'a[class="active"]', 'p[class="footer-resonances"]',
    'div[class="item-list item-list-pager"]', 'ul[class="pub-solr-sub-menu"]', 'a[href="#main-content"]', 'div[id="block-sociauxcrchum"]', 'div[class="visually-hidden"]',
    'div[id="Breadcrumb"]', 'div[id="pageInfo"]', 'div[id="breadcrumb"]', 'div[class="pagesCreation"]', 'a[href="#contenu"]', 'div[class="bandeau"]',
    'div[id="seeAlso"]', 'a[href="/nous-ecrire.aspx"]', 'li[class="CMSListMenuLI"]', 'li[class="CMSListMenuLI navFirst"]', 'li[class="CMSListMenuLI navLast"]',
    'div[class="alert alert-danger"]', 'span[class="alertoverflow"]', 'div[class="alert alert-warning alert-dismissible"]', 'ul[class="menu"]', 'div[id="letters-filter"]',
    'ul[class="pager"]', 'a[href="#main-menu"]', 'ul[class="custom_menu"]', 'h2[class="element-invisible"]', 'div[class="breadcrumb"]',
    'a[class="all-cta"]', 'div[class="sub-menu-inner container"]', 'div[class="fixed-dk-nav"]', 'div[class="fixed-dk-nav-container"]', 'div[class="container-inner"]',
    'div[class="socials"]', 'div[class="breadcrumbs"]', 'a[class="btn-print"]', 'ul[class="list-buttons"]', 'p[class="visually-hidden"]', 
    'a[class="back-to-top"]', 'a[class="sr-only sr-only-focusable"]', 'ol[class="breadcrumb"]', 'div[class="container-fluid piv_bas"]', 'div[class="col-12 formBasPage"]',
    'div[class="container-fluid rangee-footer"]', 'a[class="visuallyHidden passerContenu"]', 'div[id="bandeau-alerte"]', 'div[class="menu-sec-wrapper col-12 col-lg-12"]',
    'a[href="#layout-content"]', 'div[class="paragraph feedback"]', 'p[class="last-update"]', 'ul[class="footer__menu--list"]', 'div[class="footer__info"]',
    'section[class="hello-bar"]', 'section[class="breadcrumb"]', 'div[class="menu-page"]', 'div[class="no-print menu-non-voyant"]', 'div[class="navigation"]',
    'div[class="pure-bloc pure-u-1 pure-u-md-1-3 pure-u-lg-1-4 side-menu"]', 'div[class="pied"]', 'div[class="social"]', 'div[class="piv-bas"]',
    'div[class="partage"]', 'div[class="pied-print no-screen"]', 'div[class="carte dynamic-carte-interactive-display ui-carte-panel"]',
    'div[id="piv"]', 'ul[class="social-nav top-bar-social"]', 'div[class="sidebar"]', 'ul[class="side-menu"]', 'div[class="zoom-button-wrapper"]',
    'a[href="#maincontent"]', 'a[href="#content"]', 'p[id="breadcrumbs"]', 'div[class="mega-menu-wrap"]', 'div[class="menu_2"]', 'div[class="welcome"]',
    'div[class="header_two"]', 'div[class="footer"]', 'div[class="footer-wrapper"]', 'div[class="custom-accessibility-tools js-only"]', 'section[role="navigation"]',
    'div[class="container-fluid container-blue container-dl-menu"]', 'div[class="col-xs-12 dl-menuwrapper menu-mobile visible-x"]', 'div[class="entete-print no-screen"]']
    
    for t in tags_to_remove:
        tags = soup.find_all(t)
        for tag in tags:
            tag.decompose()

    for t in attr_to_remove:
        attr = soup.select(t)
        for a in attr:
            a.decompose()


    data = soup.get_text(separator=' # ').replace("\n", " ").replace("\r", " ") 
    data = re.compile(r"\s+").sub(" ", data).strip()
    
    return data

In [3]:
import io
from PyPDF2 import PdfReader

def getTextPDF(url):
    pdf_link = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    with io.BytesIO(pdf_link.content) as f:
        reader = PdfReader(f)
        nb_pages = len(reader.pages)
        text = ''
        if nb_pages <= 50: # On ne prendra pas les documents de plus de 50 pages
            for i in range(nb_pages):
                page = reader.pages[i]
                text += page.extract_text().lower().replace('\n', '')
        if text :
            return text


In [4]:
def scrape_list(df):
    output = []
    pdfs = df[df['Type'] == 'application/pdf']['Address'].tolist()
    htmls = df[df['Type'] == 'text/html']['Address'].tolist() 
    # Il y a aussi des pages pour lesquelles le crawler n'a pas été en mesure de consigner une valeur
    # On va essayer d'aller chercher leur contenu avec la fonction getTextHTML 
    nans = df[df["Type"].isnull()]['Address'].tolist()  
    htmls = htmls + nans

    for url in htmls :
        try: 
            text = getTextHTML(url)
            if not '���' in text:
                output.append({'Address': url, 'type': 'text/html', 'text':text.encode('utf-8', errors= 'replace').decode('utf-8')})
        except Exception as e:
            print("ERROR " + " - " + url)
            print(e)

    for url in pdfs :
        try: 
            text = getTextPDF(url)
            if not '���' in text:
                output.append({'Address': url, 'type': 'application/pdf', 'text':text.encode('utf-8', errors= 'replace').decode('utf-8')})
        except Exception as e:
            print("ERROR " + " - " + url)
            print(e)

    return output

In [5]:
path = '../03-corpus/1-crawler/'


regex = '.*png.*|.*jpeg.*|.*jpg.*|.*docx.*|.*js.*|.*font.*|.*gif.*|.*formulaire.*|.*?f%5B0%5D.*|.*img.*|.*%5Bfilter%.*|.*css.*|.*scripts.*|.*zip.*|.*xlsx.*|.*cms.*|.*/images/.*|.*sondage.*|.*/depenses/.*|.*demandes-acces.*'

# La liste des URLs à scrapper pour chaque corpus est contenue dans un fichier CSV. 
#On commence donc par lire le CSV pour extraire nos URLS.
for x in range(len(corpus_global)) : 
    acteur = corpus_global[x]['acteur']
    if acteur == 'ciusss_mcq':
        # if not pdfs:
        #   regex += '|.*pdf.*'

        # encoding= 'ISO-8859-1' "utf-8"
        with open(path + acteur + '.csv', encoding='UTF-8') as f:
            csv = read_csv(f, sep=';')
            # Nettoyer ce qui ne devrait pas se trouver là
            csv = csv[~csv["Address"].str.contains(regex, case=False)]

        fr = csv[~csv["Address"].str.contains('/en/')][['Address', 'Type']] # Données en français
        en = csv[csv["Address"].str.contains('/en/')][['Address', 'Type']] # Données en anglais
        print("On va tenter d'aspirer {} pages Web".format(len(fr) + len(en)))

        sites_fr = scrape_list(fr)
        if(len(en) > 0):
            sites_en = scrape_list(en)

        output_path = '../03-corpus/2-data/1-fr/'

        sites_fr = pd.DataFrame(sites_fr)
        sites_fr = csv.merge(sites_fr, how='right', on='Address')
        sites_fr = sites_fr[['Address', 'Title', 'Type', 'text']]

        sites_fr.to_csv(output_path + acteur + '.csv', sep=';', escapechar='/')
        corpus_global[x]['N_fr'] = len(sites_fr)
        if(len(en) > 0):
            output_path = '../03-corpus/2-data/1-en/'
            sites_en = pd.DataFrame(sites_en)
            sites_en = csv.merge(sites_en, how='right', on='Address')
            sites_en = sites_en[['Address', 'Title', 'Type', 'text']]
            sites_en.to_csv(output_path + acteur + '_en.csv', sep=';', escapechar='/')
            corpus_global[x]['N_en'] = len(sites_en)

        print('On a fini avec {}'.format(acteur))

On va tenter d'aspirer 3264 pages Web


c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanc

ERROR  - https://www.ciusssmcq.ca/nous-joindre/visioconference-clinique-pour-le-reseau-de-la-sante/
HTTPConnectionPool(host='telesantemcgill.ca', port=80): Max retries exceeded with url: /zoom/index-fr.html (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000018524B9EE00>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))


c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanc

ERROR  - https://www.ciusssmcq.ca/a-propos-de-nous/fondations/fondation-a-notre-sante-hotel-dieu-d-arthabaska/
HTTPConnectionPool(host='www.fondationhda.com', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000018524DC6080>, 'Connection to www.fondationhda.com timed out. (connect timeout=None)'))


...

ERROR  - https://www.ciusssmcq.ca/telechargement/746/recette-de-solution-de-rehydratation-maison/
argument of type 'NoneType' is not iterable


c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanc

ERROR  - https://www.ciusssmcq.ca/telechargement/239/rapport-de-recherche-sur-l-action-communautaire-en-mauricie-et-au-centre-du-queb/
argument of type 'NoneType' is not iterable


c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ciusssmcq.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ERROR  - https://www.ciusssmcq.ca/Content/Client/Librairie/Documents/10_-_Pour_un_monde_meilleur_-_Presentation_M__Jean_Proulx.pdf
argument of type 'NoneType' is not iterable
On a fini avec ciusss_mcq
